In [1]:
import os
import sys
import time
import random
import argparse
from copy import deepcopy

import pandas as pd
import numpy as np
from scipy import sparse
import bottleneck as bn

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

import wandb

In [2]:
parser = argparse.ArgumentParser(description='RecVAE for Sequential Recommendation')

parser.add_argument('--data_dir', type=str, default='../data/train/', help='Movielens train dataset path')
parser.add_argument('--min_items_per_user', type=int, default=5)
parser.add_argument('--min_users_per_item', type=int, default=0)
parser.add_argument('--heldout_users', type=int, default=3000)
parser.add_argument('--seed', type=int, default=42, help='random seed')
parser.add_argument('--cuda', action='store_true', help='use CUDA')

parser.add_argument('--hidden_dim', type=int, default=500)
parser.add_argument('--latent_dim', type=int, default=200)
parser.add_argument('--batch_size', type=int, default=500)
parser.add_argument('--beta', type=float, default=0.1)
parser.add_argument('--gamma', type=float, default=0.005)
parser.add_argument('--lr', type=float, default=2e-4)
parser.add_argument('--weight_decay', type=float, default=0)
parser.add_argument('--scheduler', type=str, default='None') 
parser.add_argument('--dropout_rate', type=float, default=0.5)
parser.add_argument('--n_epochs', type=int, default=150)
parser.add_argument('--n_enc_epochs', type=int, default=3)
parser.add_argument('--n_dec_epochs', type=int, default=1)
parser.add_argument('--not_alternating', type=bool, default=False)

exp_idx = 5
parser.add_argument('--save', type=str, default=f'./ckpts/model_exp{exp_idx}.pt',
                    help='path to save the final model')

args = parser.parse_args([])

### WandB

In [3]:
# login to WandB
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: watchstep (watchstep_). Use `wandb login --relogin` to force relogin


True

In [4]:
# initialize WandB
run = wandb.init(entity="new-recs", project="movierec", tags=["RecVAE"], group="RecVAE_watchstep", config=args)
wandb.run.name = f"RecVAE_watchstep_exp{exp_idx}"
run.save()
print(wandb.run.name)

wandb: Currently logged in as: watchstep (new-recs). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


RecVAE_watchstep_exp5


- `heldout_users` : select 3000 users as heldout user, 3000 users as validation users, and the rest of the users for training

In [5]:
# set random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available():
  torch.cuda.manual_seed(args.seed)
  args.cuda = True
  
device = torch.device('cuda:0' if args.cuda else 'cpu')
device

device(type='cuda', index=0)

### Preprocessing

In [6]:
data_dir = args.data_dir
min_uc = args.min_items_per_user
min_sc = args.min_users_per_item
n_heldout_users = args.heldout_users 

In [7]:
# https://github.com/dawenl/vae_cf
# https://github.com/ilya-shenbin/RecVAE 

# Pandas version : 2.0.2
def get_count(tp, id):
  playcount_groupbyid = tp[[id]].groupby(id)
  count = playcount_groupbyid.size()
   
  return count

# Only keep items that are clicked on by at least 5 users 
def filter_triplets(tp, min_uc=min_uc, min_sc=min_sc):
  # Only keep the triplets for items which were clicked on by at least min_sc users.
  if min_sc > 0:
    item_count = get_count(tp, 'item')
    tp = tp[tp['item'].isin(item_count.index[item_count >= min_sc])]
    
  # Only keep the triplets for users who clicked on at least min_uc items
  # After, some of the items will have less than min_uc users, but should only be a small proportion
  if min_uc > 0:
    user_count = get_count(tp, 'user')
    tp = tp[tp['user'].isin(user_count.index[user_count >= min_uc])]
  
  
  # Update both usercount and itemcount after filtering
  user_count, item_count = get_count(tp, 'user'), get_count(tp, 'item')
  return tp, user_count, item_count

#### Download data

In [8]:
raw_data = pd.read_csv(os.path.join(data_dir, 'train_ratings.csv'), header=0)
raw_data.head()

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563


In [9]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data)

In [10]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print('After filtering')
print('============================================')
print(f'{raw_data.shape[0]} events from {user_activity.shape[0]} users & {item_popularity.shape[0]} movies')
print(f'sparsity: {sparsity*100:.3f}%')

# 원본 데이터셋과 똑같은 결과 (이미 5번 이상의 리뷰가 있는 user들로만 이루어진 데이터)

After filtering
5154471 events from 31360 users & 6807 movies
sparsity: 2.415%


In [11]:
user_activity.head() # user별 리뷰수

user
11    376
14    180
18     77
25     91
31    154
dtype: int64

In [12]:
item_popularity.head() # item별 리뷰수

item
1    12217
2     3364
3      734
4       43
5      590
dtype: int64

In [13]:
unique_uid = user_activity.index
unique_uid

Index([    11,     14,     18,     25,     31,     35,     43,     50,     58,
           60,
       ...
       138459, 138461, 138470, 138471, 138472, 138473, 138475, 138486, 138492,
       138493],
      dtype='int64', name='user', length=31360)

In [14]:
# Shuffle User Indices
unique_uid = user_activity.index

unique_uid_before_shuffling = user_activity.index

print('Before shuffling')
print('================')
print(unique_uid)

idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]
print('\nAfter shuffling')
print('================')
print(unique_uid)

Before shuffling
Index([    11,     14,     18,     25,     31,     35,     43,     50,     58,
           60,
       ...
       138459, 138461, 138470, 138471, 138472, 138473, 138475, 138486, 138492,
       138493],
      dtype='int64', name='user', length=31360)

After shuffling
Index([ 81259,  11986,  67552, 127325, 115853,   5192,  93410, 124003,    422,
       115657,
       ...
        27377,  96934,  49486,  52439,  94582, 131351,  23478,   3671,  69383,
       103755],
      dtype='int64', name='user', length=31360)


In [15]:
# train/validation/test users
n_users = unique_uid.size
n_heldout_users = args.heldout_users # 10000

tr_users = unique_uid[:(n_users - n_heldout_users*2)]
vd_users = unique_uid[(n_users - n_heldout_users*2):(n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

print(f'# of all users: {n_users}')
print(f'# of train users: {len(tr_users)}')
print(f'# of validation users: {len(vd_users)}')
print(f'# of test users: {len(te_users)}')

# of all users: 31360
# of train users: 25360
# of validation users: 3000
# of test users: 3000


In [16]:
def split_train_test_proportion(data, test_prop=0.2):
  data_grouped_by_user = data.groupby('user')
  tr_list, te_list = [], []
  
  np.random.seed(args.seed)
  
  for i, (_, group) in enumerate(data_grouped_by_user):
    n_items_u = len(group)
    
    if n_items_u >= 5:
      idx = np.zeros(n_items_u, dtype='bool')
      idx[np.random.choice(n_items_u, size=int(test_prop*n_items_u), replace=False).astype('int64')] = True
      
      tr_list.append(group[np.logical_not(idx)])
      te_list.append(group[idx])
      
    else:
      tr_list.append(group)
      
    if i % 1000 == 0:
      print(f'{i} users sampled')
      sys.stdout.flush()
      
  data_tr = pd.concat(tr_list)
  data_te = pd.concat(te_list)
  
  return data_tr, data_te

def numerize(tp, profile2id, show2id):
  uid = tp['user'].apply(lambda x: profile2id[x])
  sid = tp['item'].apply(lambda x: show2id[x])
  
  return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [17]:
# train data는 전체 데이터 모두 사용
train_plays = raw_data.loc[raw_data['user'].isin(tr_users)]

unique_sid = pd.unique(train_plays['item'])

show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [18]:
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

with open(os.path.join(data_dir, 'unique_sid.txt'), 'w') as f:
  for sid in unique_sid:
    f.write(f'{sid}\n')
    
with open(os.path.join(data_dir, 'unique_uid.txt'), 'w') as f:
  for uid in unique_uid:
    f.write(f'{uid}\n')

In [19]:
# validation / test data는 input인 tr 데이터와 정답 확인하기 위한 te 데이터로 분리
vad_plays = raw_data.loc[raw_data['user'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['item'].isin(unique_sid)]

vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

test_plays = raw_data.loc[raw_data['user'].isin(te_users)]
test_plays = test_plays.loc[test_plays['item'].isin(unique_sid)]

test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
1000 users sampled
2000 users sampled


0 users sampled
1000 users sampled
2000 users sampled


#### Save data into (user_index, item_index) format

In [20]:
train_data = numerize(train_plays, profile2id, show2id)
# train_data.to_csv(os.path.join(data_dir, 'train.csv'), index=False)

vad_data_tr = numerize(vad_plays_tr, profile2id, show2id)
# vad_data_tr.to_csv(os.path.join(data_dir, 'validation_tr.csv'), index=False)

vad_data_te = numerize(vad_plays_te, profile2id, show2id)
# vad_data_te.to_csv(os.path.join(data_dir, 'validation_te.csv'), index=False)

test_data_tr = numerize(test_plays_tr, profile2id, show2id)
# test_data_tr.to_csv(os.path.join(data_dir, 'test_tr.csv'), index=False)

test_data_te = numerize(test_plays_te, profile2id, show2id)
# test_data_te.to_csv(os.path.join(data_dir, 'test_te.csv'), index=False)

# # uid sorting for submission (before shuffling )
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid_before_shuffling))
inference_data = numerize(raw_data, profile2id, show2id)
# inference_data.to_csv(os.path.join(data_dir, 'inference.csv'), index=False)

In [21]:
train_data.head(3)

,uid,sid
0,13266,0
1,13266,1
2,13266,2


In [22]:
vad_data_tr.head(3)

,uid,sid
2133,28125,887
2134,28125,1162
2135,28125,1418


In [23]:
test_data_tr.head(3)

,uid,sid
1514,30380,468
1515,30380,1717
1516,30380,2449


In [24]:
inference_data.head(3)

,uid,sid
0,0,0
1,0,1
2,0,2


#### Data Loader

In [25]:
class DataLoader():
  '''
  Load Movielens dataset for RecVAE
  ''' 
  def __init__(self, data_dir):
    self.data_dir = data_dir
    assert os.path.exists(data_dir), "DATA NOT EXIT"
    
    self.n_items = self.load_n_items()
  
  def load_data(self, data_type='train'):
    if data_type == 'train':
      return self._load_train_data(data_type)
    elif data_type == 'validation':
      return self._load_tr_te_data(data_type)
    elif data_type == 'test':
      return self._load_tr_te_data(data_type)
    elif data_type == 'inference':
      return self._load_train_data(data_type)
    else:
      raise ValueError('datatype should be in [train, validation, test, inference]')
  
  def load_items(self):
    unique_sid = np.loadtxt(os.path.join(self.data_dir, 'unique_sid.txt'))
    return unique_sid
  
  def load_n_items(self):
    unique_sid = []
    with open(os.path.join(self.data_dir, 'unique_sid.txt'), 'r') as f:
      for line in f:
        unique_sid.append(line.strip())
        
    n_items = len(unique_sid)
    return n_items
  
  def load_n_users(self):
    unique_uid = []
    with open(os.path.join(self.data_dir, 'unique_uid.txt'), 'r') as f:
      for line in f:
        unique_uid.append(line.strip())
    
    n_users = len(unique_uid)
    return n_users
  
  def _load_train_data(self, data_type='train'):
    path = os.path.join(self.data_dir, '{}.csv'.format(data_type))
    
    tp = pd.read_csv(path)
    n_users = tp['uid'].max() + 1
    
    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, self.n_items))
    
    return data
  
  def _load_tr_te_data(self, data_type='test'):
    tr_path = os.path.join(self.data_dir, '{}_tr.csv'.format(data_type))
    te_path = os.path.join(self.data_dir, '{}_te.csv'.format(data_type))

    tp_tr = pd.read_csv(tr_path)
    tp_te = pd.read_csv(te_path)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                                (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                                (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, self.n_items))
    
    return data_tr, data_te

### Define Model

<img src ="https://www.researchgate.net/profile/Elena-Tutubalina/publication/338158079/figure/fig1/AS:839837569540096@1577244288014/RecVAE-architecture.ppm" width='400'>

In [26]:
def swish(x):
  return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
  return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())

class CompositePrior(nn.Module):
  def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
    super(CompositePrior, self).__init__()
    
    self.mixture_weights = mixture_weights
    
    self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
    self.mu_prior.data.fill_(0)
    
    self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
    self.logvar_prior.data.fill_(0)
    
    self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
    self.logvar_uniform_prior.data.fill_(10)
    
    self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
    self.encoder_old.requires_grad_(False)
    
  def forward(self, x, z):
    post_mu, post_logvar = self.encoder_old(x, 0)
    
    # encoder의 output과 이전 epoch의 파라미터를 지정한 encoder (as a pior)간의 KL Divergence
    stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
    post_prior = log_norm_pdf(z, post_mu, post_logvar)
    unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)
    
    gaussians = [stnd_prior, post_prior, unif_prior]
    gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]
    
    density_per_gaussian = torch.stack(gaussians, dim=-1)
    
    return torch.logsumexp(density_per_gaussian, dim=-1)

class Encoder(nn.Module):
  def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
    super(Encoder, self).__init__()
    
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
    self.fc2 = nn.Linear(hidden_dim, hidden_dim)
    self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
    self.fc3 = nn.Linear(hidden_dim, hidden_dim)
    self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
    self.fc4 = nn.Linear(hidden_dim, hidden_dim)
    self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
    self.fc5 = nn.Linear(hidden_dim, hidden_dim)
    self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
    self.fc_mu = nn.Linear(hidden_dim, latent_dim)
    self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
    
  def forward(self, x, dropout_rate):
    norm = x.pow(2).sum(dim=-1).sqrt()
    x = x / norm[:, None]
    
    x = F.dropout(x, p=dropout_rate, training=self.training)
    
    h1 = self.ln1(swish(self.fc1(x)))
    h2 = self.ln2(swish(self.fc2(h1) + h1))
    h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
    h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
    h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
    
    return self.fc_mu(h5), self.fc_logvar(h5)

In [27]:
class RecVAE(nn.Module):
  def __init__(self, hidden_dim, latent_dim, input_dim):
    super(RecVAE, self).__init__()
    
    self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
    self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
    self.decoder = nn.Linear(latent_dim, input_dim)
    
  def reparameterize(self, mu, logvar):
    if self.training:
      std = torch.exp(0.5*logvar)
      eps = torch.randn_like(std)
      return eps.mul(std).add_(mu)
    else:
      return mu
  
  def forward(self, user_ratings, beta=None, gamma=1, dropout_rate=0.5, calculate_loss=True):
    mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)
    z = self.reparameterize(mu, logvar)
    x_pred = self.decoder(z)
    
    if calculate_loss:
      if gamma:
        norm = user_ratings.sum(dim=-1)
        kl_weight = gamma*norm
      elif beta:
        kl_weight = beta
      
      mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
      kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
      negative_elbo = -(mll - kld)
      
      return (mll, kld), negative_elbo
    
    else:
      return x_pred
  
  def update_prior(self):
    self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))

### Metric

In [28]:
def ndcg(X_pred, heldout_batch, k=100):
  '''
  normalized discounted cumulative gain@k for binary relevance
  ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
  '''
  batch_users = X_pred.shape[0]
  idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
  topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                     idx_topk_part[:, :k]]
  idx_part = np.argsort(-topk_part, axis=1)
  
  idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
  
  tp = 1. / np.log2(np.arange(2, k+2))
  
  DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                       idx_topk].toarray() * tp).sum(axis=1)
  IDCG = np.array([(tp[:min(n, k)]).sum()
                   for n in heldout_batch.getnnz(axis=1)])
  return DCG / IDCG


def recall(X_pred, heldout_batch, k=100):
  batch_users = X_pred.shape[0]
  
  idx = bn.argpartition(-X_pred, k, axis=1)
  X_pred_binary = np.zeros_like(X_pred, dtype=bool)
  X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True
  
  X_true_binary = (heldout_batch > 0).toarray()
  tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(np.float32)
  
  recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
  
  return recall

### Train

In [29]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
  assert 0 < samples_perc_per_epoch <= 1
  
  N = data_in.shape[0]
  samples_per_epoch = int(N * samples_perc_per_epoch)
  
  if shuffle:
    idx_list = np.arange(N)
    np.random.shuffle(idx_list)
    idx_list = idx_list[:samples_per_epoch]
  else:
    idx_list = np.arange(samples_per_epoch)
  
  for st_idx in range(0, samples_per_epoch, batch_size):
    end_idx = min(st_idx + batch_size, samples_per_epoch)
    idx = idx_list[st_idx:end_idx]
    
    yield Batch(device, idx, data_in, data_out)
    
class Batch:
  def __init__(self, device, idx, data_in, data_out=None):
    self._device = device
    self._idx = idx
    self._data_in = data_in
    self._data_out = data_out
    
  def get_idx(self):
    return self._idx
  
  def get_idx_to_dev(self):
    return torch.LongTensor(self.get_idx()).to(self._device)
  
  def get_ratings(self, is_out=False):
    data = self._data_out if is_out else self._data_in
    return data[self._idx]
  
  def get_ratings_to_dev(self, is_out=False):
    return torch.Tensor(
      self.get_ratings(is_out).toarray()
    ).to(self._device)

In [30]:
def train(model, opts, train_data, batch_size, n_epochs, beta, gamma, dropout_rate):
  # training mode
  model.train()
  
  for epoch in range(n_epochs):
    for batch in generate(batch_size, device, data_in=train_data, shuffle=True):
      ratings = batch.get_ratings_to_dev()
      
      for optimizer in opts:
        optimizer.zero_grad()
      
      _, loss = model(ratings, beta, gamma, dropout_rate)
      loss.backward()
      
      for optimizer in opts:
        optimizer.step()
      
    # for scheduler in sches:
    #   scheduler.step()

In [31]:
def evaluate(model, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=500):
  # evaluation mode
  model.eval()
  metrics = deepcopy(metrics)
  
  for m in metrics:
    m['score'] = []
    
  for batch in generate(batch_size, 
                        device, 
                        data_in, 
                        data_out,
                        samples_perc_per_epoch):
    
    items_in = batch.get_ratings_to_dev()
    items_out = batch.get_ratings(is_out=True)
    
    items_pred = model(items_in, calculate_loss=False).cpu().detach().numpy()
    
    if not(data_in is data_out):
      items_pred[batch.get_ratings().nonzero()] = -np.inf
      
    for m in metrics:
      m['score'].append(m['metric'](items_pred, items_out, k=m['k']))
      
  
  for m in metrics:
    m['score'] = np.concatenate(m['score']).mean()
    
  return [x['score'] for x in metrics]

In [32]:
# load data
loader = DataLoader(args.data_dir)

n_items = loader.load_n_items()
n_users = loader.load_n_users()

train_data = loader.load_data('train')
vad_data_tr, vad_data_te = loader.load_data('validation')
test_data_tr, test_data_te = loader.load_data('test')

In [33]:
model_kwargs = {
  'hidden_dim': args.hidden_dim,
  'latent_dim': args.latent_dim,
  'input_dim': train_data.shape[1]
}

metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = RecVAE(**model_kwargs).to(device)
model_best = RecVAE(**model_kwargs).to(device)

learning_kwargs = {
  'model': model,
  'train_data': train_data,
  'batch_size': args.batch_size,
  'beta': args.beta,
  'gamma': args.gamma
}

decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

# https://pytorch.org/docs/stable/optim.html
optimizer_encoder = optim.Adam(encoder_params, lr=args.lr, weight_decay=args.weight_decay)
optimizer_decoder = optim.Adam(decoder_params, lr=args.lr, weight_decay=args.weight_decay)
# scheduler_encoder = optim.lr_scheduler.CosineAnnealingLR(optimizer_encoder, T_max=50, eta_min=0.0001)
# scheduler_decoder = optim.lr_scheduler.CosineAnnealingLR(optimizer_decoder, T_max=50, eta_min=0.0001)

In [34]:
for epoch in range(args.n_epochs):
  if args.not_alternating:
    train(opts=[optimizer_encoder, optimizer_decoder], epochs=1, dropout_rate=args.dropout_rate, **learning_kwargs)
  else:
    train(opts=[optimizer_encoder], n_epochs=args.n_enc_epochs, dropout_rate=args.dropout_rate, **learning_kwargs)
    model.update_prior()
    train(opts=[optimizer_decoder], n_epochs=args.n_dec_epochs, dropout_rate=0, **learning_kwargs)
  
  train_scores.append(
    evaluate(model, train_data, train_data, metrics, 0.01)[0]
  )
  
  valid_scores.append(
    evaluate(model, vad_data_tr, vad_data_te, metrics, 1)[0]
  )
  
  if valid_scores[-1] > best_ndcg:
    best_ndcg = valid_scores[-1]
    model_best.load_state_dict(deepcopy(model.state_dict()))
    with open(args.save, 'wb') as f:
      torch.save(model, f)
    
  print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
        f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')
  
  if (epoch % 10) == 0:
    wandb.log({"best valid": best_ndcg, "valid ndcg@100": valid_scores[-1], "train ndcg@100": train_scores[-1]})

# test data  
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_data_tr, test_data_te, test_metrics)

for metric, score in zip(test_metrics, final_scores):
  print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

wandb.log({"test ndcg@100": final_scores[0], "test recall@20": final_scores[1], "test recall@50": final_scores[2]})

epoch 0 | valid ndcg@100: 0.1990 | best valid: 0.1990 | train ndcg@100: 0.3842
epoch 1 | valid ndcg@100: 0.3140 | best valid: 0.3140 | train ndcg@100: 0.5074
epoch 2 | valid ndcg@100: 0.3562 | best valid: 0.3562 | train ndcg@100: 0.5637
epoch 3 | valid ndcg@100: 0.3815 | best valid: 0.3815 | train ndcg@100: 0.5983
epoch 4 | valid ndcg@100: 0.3950 | best valid: 0.3950 | train ndcg@100: 0.6178
epoch 5 | valid ndcg@100: 0.4050 | best valid: 0.4050 | train ndcg@100: 0.6313
epoch 6 | valid ndcg@100: 0.4119 | best valid: 0.4119 | train ndcg@100: 0.6412
epoch 7 | valid ndcg@100: 0.4174 | best valid: 0.4174 | train ndcg@100: 0.6494
epoch 8 | valid ndcg@100: 0.4231 | best valid: 0.4231 | train ndcg@100: 0.6560
epoch 9 | valid ndcg@100: 0.4269 | best valid: 0.4269 | train ndcg@100: 0.6608
epoch 10 | valid ndcg@100: 0.4306 | best valid: 0.4306 | train ndcg@100: 0.6655
epoch 11 | valid ndcg@100: 0.4331 | best valid: 0.4331 | train ndcg@100: 0.6692
epoch 12 | valid ndcg@100: 0.4355 | best valid: 0.

## Inference

In [ ]:
# load the best saved model.
with open(args.save, 'rb') as f:
  model = torch.load(f)
  
loader = DataLoader(args.data_dir)

# load inference data
inference_data = loader.load_data('inference')

n_items = loader.load_n_items() # train_data.shape[1]
n_users = loader.load_n_users() # train_data.shape[0]

print(f'# of items: {n_items}')
print(f'# of users: {n_users}')

# of items: 6807
# of users: 31360


In [ ]:
def inference(model, data_in, samples_perc_per_epoch=1, batch_size=500):
  model.eval()
  output = []
  
  with torch.no_grad():
    for batch in generate(batch_size, 
                          device,
                          data_in,
                          samples_perc_per_epoch):
      
      ratings_in = batch.get_ratings_to_dev()
      
      ratings_pred = model(ratings_in, calculate_loss=False).detach().cpu().numpy()
      
      # remove watched items
      ratings_pred[batch.get_ratings().nonzero()] = -np.inf
      
      n_users = ratings_pred.shape[0]
      for i in range(n_users):
        output.append(ratings_pred[i])
        
  return output

In [ ]:
inference_output = inference(model, inference_data)

submission = []

for idx in range(n_users):
  # item descending order
  sid_idx_preds_per_user = np.argsort(inference_output[idx])[::-1]
  sid_preds = unique_sid[sid_idx_preds_per_user]
  
  tmp = []
  for item in sid_preds:
    if len(tmp) == 10:
      break
    tmp.append((unique_uid_before_shuffling[idx], item))
  
  submission.extend(tmp)

In [ ]:
submission_df = pd.DataFrame(submission, columns=['user', 'item'])
submission_df.to_csv(f'./submissions/recvae_submission_exp{exp_idx}.csv', index=False)

In [ ]:
submission_df.head()

,user,item
0,11,4370
1,11,37386
2,11,58025
3,11,7373
4,11,4886


In [ ]:
# Finish WandB run
run.finish()

best valid,▁▇██████████████████
test ndcg@100,▁
test recall@20,▁
test recall@50,▁
train ndcg@100,▁▆▇▇▇▇▇▇▇▇██████████
valid ndcg@100,▁▇██████████████████
best valid,0.46919
test ndcg@100,0.47417
test recall@20,0.37695
test recall@50,0.43635
train ndcg@100,0.77196
